# Pittsburgh-area aviation waypoints

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import re
import requests

### Fetch raw waypoints from the FAA

In [2]:
waypoints = []

# First 1,000 waypoints
search_data = {
    'dataType': 'LIDFIXESWAYPOINTS',
    'start': 0,
    'length': 1000,
    'sortcolumn': 'fix_identifier',
    'sortdir': 'asc',
    'searchval': 'PENNSYLVANIA',
    'r': 'NTAg7a9ZdvAPgPALayuxIQYO',
}

response = requests.post('https://nfdc.faa.gov/nfdcApps/controllers/PublicDataController/getLidData', data=search_data)
response = response.json()
waypoints = waypoints + response['data']

# Next 1,000 waypoints
search_data['start'] = 1000
response = requests.post('https://nfdc.faa.gov/nfdcApps/controllers/PublicDataController/getLidData', data=search_data)
response = response.json()
waypoints = waypoints + response['data']

In [3]:
print([response['totalrows'], len(waypoints)])

[1651, 1651]


### Convert raw waypoints to parsed waypoints

In [4]:
for waypoint in waypoints:
    coords = waypoint['description'].split(' ')[-2:]

    lat = re.findall(r'(\d+)-(\d+)-(\d+).(\d+)N', coords[0])
    lat = (float(lat[0][0])) + (float(lat[0][1]) / 60.0) + ((float(lat[0][2]) + (.0001 * float(lat[0][3]))) / 3600)

    lon = re.findall(r'(\d+)-(\d+)-(\d+).(\d+)W', coords[1])
    lon = (float(lon[0][0])) + (float(lon[0][1]) / 60) + ((float(lon[0][2]) + (.0001 * float(lon[0][3]))) / 3600)

    waypoint['coords'] = [-1 * lon, lat]

### Filter waypoints to be within metro Pittsburgh

In [5]:
# South of MYRON
south_of = [w for w in waypoints if w['fix_identifier'] == 'MYRON'][0]['coords'][1]
waypoints = [w for w in waypoints if w['coords'][1] <= south_of]

# North of CRSBY
north_of = [w for w in waypoints if w['fix_identifier'] == 'CRSBY'][0]['coords'][1]
waypoints = [w for w in waypoints if w['coords'][1] >= north_of]

# West of Trafford
waypoints = [w for w in waypoints if w['coords'][0] < -79.8] 

In [6]:
len(waypoints)

103

### Convert waypoints to dataframe

In [7]:
df = gpd.GeoDataFrame({
    'identifier': [w['fix_identifier'] for w in waypoints],
    'geometry': [Point(w['coords']) for w in waypoints]
})

In [8]:
df.to_file('points.json', driver='GeoJSON')